In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence
import math
import torch
import numpy as np
from random import *
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [ ]:
#input dataset
X = pd.read_csv("../input/the-movies-dataset/movies_metadata.csv")
overview = X['overview']
name_overview = X[['original_title','overview','id']]

#prepare the corpus
sentence=[]
for i in overview:
    sentence.append(text_to_word_sequence(str(i),filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n',lower=True,split=" "))
sentences = [" ".join(i) for i in sentence]

In [ ]:
pip install sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
#get pretrained BERT model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

In [ ]:
tokens = {'input_ids': [], 'attention_mask': []}
for sentence in sentences:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [ ]:
tokens['attention_mask'][0:1].shape

In [ ]:
#use average hidden output of the last layer
senvec = []
for i in range(len(sentences)):
    if i%20 == 19:
        print('Iter: ',i+1)

    outputs =  model(tokens['input_ids'][i:i+1],tokens['attention_mask'][i:i+1])
    
    #use the hidden output of the last layer as the sentence embedding
    embeddings = outputs.last_hidden_state
    #get correspndng masks
    attention_mask = tokens['attention_mask'][i:i+1]
    mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    
    #get masked embedding
    masked_embeddings = embeddings * mask
    
    summed = torch.sum(masked_embeddings, 1)
    
    #sum the number of values that must be given attention
    summed_mask = torch.clamp(mask.sum(1), min=1e-9)
    
    #calculate the mean as the sum of the embedding divided by the number of values that should be given attention
    
    mean_pooled = summed / summed_mask
    
    mean_pooled = mean_pooled.detach().numpy()
    
    senvec.append(mean_pooled)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
senvec[0].shape

In [ ]:
senvec_ = np.array(senvec)
np.save('senvec_bert.npy',senvec_) 

In [ ]:
similarity_movie('Batman',10,name_overview,senvec)